In [1]:
#main_taberconley_controls.do
#sample.do
#!pip install linearmodels
#!pip install econtools
import pandas as pd
import numpy as np
import matplotlib as plt
from linearmodels import PanelOLS
import statsmodels.api as sm
import econtools as econ
import econtools.metrics as mt

df = pd.read_stata('data/Authority.dta')
print(df.shape)
#list(df.columns)

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

#for statement check            
#df.head
#print(max(df['id_path']))

(16127, 31)


In [2]:
#keep the observations need
df5= df[((df['turin_co_sample'] == 1)|(df['turin_pr_sample']==1))&(df['post_experience']>=5)&(df['pre_experience']>=5)&(df['missing']==0)]
df5= df5[(df5['ctrl_pop_turin_co_sample']==1) | (df5['ctrl_pop_turin_pr_sample']==1) | (df5['ctrl_exp_turin_co_sample']==1) | (df5['ctrl_exp_turin_pr_sample']==1) | (df5['ctrl_pop_exp_turin_co_sample']==1) | (df['ctrl_pop_exp_turin_pr_sample']==1)]

C:\Users\thxkn\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
#reconstruct trend_pa
df['id_auth_remained'] = df['id_auth']
id_auth_remained_dum = pd.get_dummies(df5['id_auth']).rename(columns=lambda x: 'id_auth_remained' + str(x))

df = pd.concat([df, id_auth_remained_dum],axis = 1)

for i in range(len(auth_list)):
    df['trend_pa_remained'+str(i+1)] =0
    if df['id_auth_remained'+str(i+1)]==1 and (df['authority_code']!= 3090272) and (df['authority_code']!=3070001):
        df.loc[i ,'trend_pa_remained'+str(i+1)] = df.loc[i, 'trend']

KeyError: 'id_auth_remained1'

In [ ]:
#create dummies

c_outcomes = 1
i = 5

for t in treatment:
    for g in group:
        for o in outcomes:
            
            #matrix = nc_ + o + t 
            df_tab5 = df[(df[t]==1) & (df[g +'_' + t]==1) & (df['post_experience']>=i) & (df['pre_experience']>=i) & (df['post_experience'].isnull()==False) & (df['pre_experience'].isnull()==False) & (df['missing']==0) & (df[o].isnull()==False) & (df['fiscal_efficiency'].isnull()==False) & (df['reserve_price'].isnull()==False) & (df['municipality'].isnull()=False) & (df['work_category']!="")]
            
            

In [ ]:
#create dummies for administration-year pairs 

all_years = df['year'].unique()
all_authorities = df['authority_code'].unique
for auth in all_authorities:
    for yr in all_ywars:
        df['auth_year_' + auth + yr] = 0
        df.loc[(df['year']==yr) & (df['authority_code']==auth), 'auth_year_' + auth + yr ] = 1

In [ ]:
# create dummies for work category
from statsmodels.stats.outliers_influence import variance_inflation_factor

all_categories = df['work_cateogory'].unique()
i = 1
for cat in all_categories:
    df['cat_'+cat] = 0
    df.loc[df['work_category']==cat, 'cat'+cat] =1
    # Regression first stage 
    
    # mutlicollinearity function
    def calc_vif(X):

        # Calculating VIF
        vif = pd.DataFrame()
        vif["variables"] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

        return(vif)

    #iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    #exog.remove('municipality')

    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg) 

    #미쳤다 코드 겁나 기네...